# Face Recognition 
> Abhishek Swami
> Roll No. 13

In [ ]:
import cv2
import os

# Define the dataset directory
dataset_directory = 'face_dataset'
os.makedirs(dataset_directory, exist_ok=True)
user=input("Enter your name: ")

# Load the pre-trained Haar Cascade classifier for face detection
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

# Initialize video capture from the live camera
video_capture = cv2.VideoCapture(0)  # Use 0 for the default camera

# Define a counter for the face images
face_counter = 0

print("Press 's' to save a face image, or 'q' to quit the program.")

# Start the video capture loop
while True:
    # Capture a frame from the video stream
    ret, frame = video_capture.read()
    
    if not ret:
        print("Failed to capture video frame.")
        break
    
    # Convert the frame to grayscale
    gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    
    # Detect faces in the frame using the Haar Cascade classifier
    faces = face_cascade.detectMultiScale(gray_frame, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))
    
    # Draw rectangles around detected faces
    for (x, y, w, h) in faces:
        cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)
    
    # Display the frame with detected faces
    cv2.imshow('Face Detection', frame)
    
    # Check for user input
    key = cv2.waitKey(1) & 0xFF
    
    # If the user presses the 's' key, save the detected faces
    if key == ord('s'):
        for (x, y, w, h) in faces:
            # Crop the face region from the frame
            face_image = frame[y:y + h, x:x + w]
            
            # Save the face image to the dataset directory
            face_image_path = os.path.join(dataset_directory, f'{user}_{face_counter}.jpg')
            cv2.imwrite(face_image_path, face_image)
            print(f"Saved face image: {face_image_path}")
            
            # Increment the face counter
            face_counter += 1
    
    # If the user presses the 'q' key, exit the loop
    if key == ord('q'):
        print("Exiting program.")
        break

# Release the video capture object and close the window
video_capture.release()
cv2.destroyAllWindows()


In [ ]:
import numpy as np
import keras
from keras import layers, models
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelBinarizer
from sklearn.metrics import classification_report

# Define the dataset directory
dataset_directory = 'face_dataset'
labels_directory = 'labels'
os.makedirs(dataset_directory, exist_ok=True)
os.makedirs(labels_directory, exist_ok=True)

# Load the dataset and preprocess it
def load_dataset():
    face_images = []
    labels = []

    # Iterate over the face images in the dataset directory
    for filename in os.listdir(dataset_directory):
        if filename.endswith('.jpg'):
            # Load the face image
            face_image_path = os.path.join(dataset_directory, filename)
            face_image = cv2.imread(face_image_path)
            
            # Resize the face image to a consistent size (e.g., 64x64)
            face_image = cv2.resize(face_image, (64, 64))
            
            # Normalize the image
            face_image = face_image.astype('float32') / 255.0
            
            # Add the face image to the list
            face_images.append(face_image)
            
            # Extract the label from the file name (assuming it's part of the filename)
            label = filename.split('_')[0]
            labels.append(label)
    
    return np.array(face_images), labels

# Load the dataset
face_images, labels = load_dataset()

# Convert labels to one-hot encoding
label_binarizer = LabelBinarizer()
labels = label_binarizer.fit_transform(labels)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(face_images, labels, test_size=0.2, random_state=42)

# Define the ANN model architecture
def create_ann_model():
    model = models.Sequential()
    model.add(layers.Flatten(input_shape=(64, 64, 3)))  # Input layer
    model.add(layers.Dense(128, activation='relu'))  # Hidden layer
    model.add(layers.Dense(128, activation='relu'))  # Another hidden layer
    model.add(layers.Dense(len(label_binarizer.classes_), activation='softmax'))  # Output layer
    
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    
    return model

# Create the ANN model
model = create_ann_model()

# Train the ANN model
history = model.fit(X_train, y_train, epochs=100, batch_size=32, validation_data=(X_test, y_test))

# Evaluate the model
print("\nModel Evaluation:")
test_loss, test_accuracy = model.evaluate(X_test, y_test)
print(f"Test Loss: {test_loss:.4f}, Test Accuracy: {test_accuracy:.4f}")

# Print classification report
y_pred = model.predict(X_test)
y_pred_classes = np.argmax(y_pred, axis=1)
y_test_classes = np.argmax(y_test, axis=1)
print("\nClassification Report:")
print(classification_report(y_test_classes, y_pred_classes, target_names=label_binarizer.classes_))

# Save the model for future use
model.save('face_recognition_ann_model.keras')
print("\nModel saved!")


In [ ]:

# Define a function to recognize faces in live camera feed using the trained ANN model
def recognize_faces_live():
    # Load the trained ANN model
    model = keras.models.load_model('face_recognition_ann_model.keras')

    # Load the Haar Cascade classifier for face detection
    face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

    # Initialize video capture from the live camera
    video_capture = cv2.VideoCapture(0)  # Use 0 for the default camera

    print("\nPress 'q' to quit the program.")

    # Start the video capture loop
    while True:
        # Capture a frame from the video stream
        ret, frame = video_capture.read()
        
        if not ret:
            print("Failed to capture video frame.")
            break
        
        # Convert the frame to grayscale
        gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        
        # Detect faces in the frame using the Haar Cascade classifier
        faces = face_cascade.detectMultiScale(gray_frame, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))
        
        # Recognize faces using the trained ANN model
        for (x, y, w, h) in faces:
            # Extract the face region from the frame
            face_region = frame[y:y + h, x:x + w]
            
            # Resize the face region to the input size of the model (64x64)
            face_region_resized = cv2.resize(face_region, (64, 64))
            
            # Normalize the face region
            face_region_normalized = face_region_resized.astype('float32') / 255.0
            
            # Add a batch dimension and make predictions using the model
            face_region_input = np.expand_dims(face_region_normalized, axis=0)
            prediction = model.predict(face_region_input)
            
            # Get the predicted class and label
            predicted_class = np.argmax(prediction, axis=1)[0]
            predicted_label = label_binarizer.classes_[predicted_class]
            
            # Draw a rectangle around the detected face
            cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)
            
            # Display the predicted label above the face
            cv2.putText(frame, predicted_label, (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 2, (0, 0, 255), 3)
        
        # Display the frame with detected faces and predicted labels
        cv2.imshow('Face Recognition', frame)
        
        # Check for user input
        key = cv2.waitKey(1) & 0xFF
        
        # If the user presses the 'q' key, exit the loop
        if key == ord('q'):
            print("Exiting program.")
            break

    # Release the video capture object and close the window
    video_capture.release()
    cv2.destroyAllWindows()

# Recognize faces in live camera feed using the trained ANN model
recognize_faces_live()
